# Introduction

In this project, you will build a neural network of your own design to evaluate the CIFAR-10 dataset.

To meet the requirements for this project, you will need to achieve an accuracy greater than 45%. 
If you want to beat Detectocorp's algorithm, you'll need to achieve an accuracy greater than 70%. 
(Beating Detectocorp's algorithm is not a requirement for passing this project, but you're encouraged to try!)

Some of the benchmark results on CIFAR-10 include:

78.9% Accuracy | [Deep Belief Networks; Krizhevsky, 2010](https://www.cs.toronto.edu/~kriz/conv-cifar10-aug2010.pdf)

90.6% Accuracy | [Maxout Networks; Goodfellow et al., 2013](https://arxiv.org/pdf/1302.4389.pdf)

96.0% Accuracy | [Wide Residual Networks; Zagoruyko et al., 2016](https://arxiv.org/pdf/1605.07146.pdf)

99.0% Accuracy | [GPipe; Huang et al., 2018](https://arxiv.org/pdf/1811.06965.pdf)

98.5% Accuracy | [Rethinking Recurrent Neural Networks and other Improvements for ImageClassification; Nguyen et al., 2020](https://arxiv.org/pdf/2007.15161.pdf)

Research with this dataset is ongoing. Notably, many of these networks are quite large and quite expensive to train. 

## Imports

In [1]:
## This cell contains the essential imports you will need – DO NOT CHANGE THE CONTENTS! ##
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torchvision import datasets
import torchvision.models as models

## Load the Dataset

Specify your transforms as a list first.
The transforms module is already loaded as `transforms`.

CIFAR-10 is fortunately included in the torchvision module.
Then, you can create your dataset using the `CIFAR10` object from `torchvision.datasets` ([the documentation is available here](https://pytorch.org/docs/stable/torchvision/datasets.html#cifar)).
Make sure to specify `download=True`! 

Once your dataset is created, you'll also need to define a `DataLoader` from the `torch.utils.data` module for both the train and the test set.

In [38]:
# Define transforms
## YOUR CODE HERE ##
transform = transforms.Compose([transforms.Resize(256),
                                transforms.CenterCrop(224),
                                transforms.ToTensor()])

train_transform = transforms.Compose([transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], 
                                                           [0.229, 0.224, 0.225])])

test_transform = transforms.Compose([transforms.ToTensor(),
                                     transforms.Normalize([0.485, 0.456, 0.406], 
                                                          [0.229, 0.224, 0.225])])

# Create training set and define training dataloader
## YOUR CODE HERE ##
train_datasets = datasets.CIFAR10(root='./data', train=True,download=True,transform=train_transform)
train_dataloaders = torch.utils.data.DataLoader(train_datasets, batch_size=64,shuffle=True,num_workers=2)

# Create test set and define test dataloader
## YOUR CODE HERE ##
test_datasets = datasets.CIFAR10(root='./data', train=False,download=True, transform=test_transform)
test_dataloaders = torch.utils.data.DataLoader(test_datasets, batch_size=64, shuffle=False, num_workers=2)

# The 10 classes in the dataset
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


## Explore the Dataset
Using matplotlib, numpy, and torch, explore the dimensions of your data.

You can view images using the `show5` function defined below – it takes a data loader as an argument.
Remember that normalized images will look really weird to you! You may want to try changing your transforms to view images.
Typically using no transforms other than `toTensor()` works well for viewing – but not as well for training your network.
If `show5` doesn't work, go back and check your code for creating your data loaders and your training/test sets.

In [39]:
def show5(img_loader):
    dataiter = iter(img_loader)
    
    batch = next(dataiter)
    labels = batch[1][0:5]
    images = batch[0][0:5]
    for i in range(5):
        print(classes[labels[i]])
    
        image = images[i].numpy()
        plt.imshow(np.rot90(image.T, k=3))
        plt.show()

## Build your Neural Network
Using the layers in `torch.nn` (which has been imported as `nn`) and the `torch.nn.functional` module (imported as `F`), construct a neural network based on the parameters of the dataset. 
Feel free to construct a model of any architecture – feedforward, convolutional, or even something more advanced!

In [46]:
## YOUR CODE HERE ##


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(3072, 405)
        self.fc2 = nn.Linear(405, 64)
        
        
        self.dropout = nn.Dropout(p=0.2)
        
       
        
    def forward(self, x):
        # make sure input tensor is flattened
        x = x.view(x.shape[0], -1)
        
        x = self.dropout(F.relu(self.fc1(x)))
        
        x = F.log_softmax(self.fc2(x), dim=1)



model = Network()
print(model)


Network(
  (fc1): Linear(in_features=3072, out_features=405, bias=True)
  (fc2): Linear(in_features=405, out_features=64, bias=True)
  (dropout): Dropout(p=0.2)
)


Specify a loss function and an optimizer, and instantiate the model.

If you use a less common loss function, please note why you chose that loss function in a comment.

In [47]:
## YOUR CODE HERE ##


criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
model.to(device)





Network(
  (fc1): Linear(in_features=3072, out_features=405, bias=True)
  (fc2): Linear(in_features=405, out_features=64, bias=True)
  (dropout): Dropout(p=0.2)
)

## Running your Neural Network
Use whatever method you like to train your neural network, and ensure you record the average loss at each epoch. 
Don't forget to use `torch.device()` and the `.to()` method for both your model and your data if you are using GPU!

If you want to print your loss during each epoch, you can use the `enumerate` function and print the loss after a set number of batches. 250 batches works well for most people!

In [48]:
epochs = 3
steps = 0
print_every = 60

train_losses, valid_losses = [], []
for e in range(epochs):
    running_loss = 0
    for images, labels in train_dataloaders:
        steps += 1
        images, labels = images.to(device), labels.to(device)
        log_ps = model.forward(images)
        loss = criterion(log_ps, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if steps%print_every==0:
            validate_loss = 0
            accuracy = 0
            model.eval()
        
            with torch.no_grad():
           
                for images, labels in test_dataloaders:
                    images, labels = images.to(device), labels.to(device)
                    log_ps = model(images)
                    validate_loss += criterion(log_ps, labels)
                
                    ps = torch.exp(log_ps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor))
        

            model.train()
            train_losses.append(running_loss/len(train_dataloaders))
            valid_losses.append(validate_loss/len(test_dataloaders))
            
print("Epoch: {}/{}.. ".format(e+1, epochs),
                    "Training Loss: {:.3f}.. ".format(running_loss/len(train_dataloaders)),
                    "Validate Loss: {:.3f}.. ".format(test_loss/len(test_dataloaders)),
                    "Validate Accuracy: {:.3f}".format(accuracy/len(test_dataloaders)))
        
            
              

AttributeError: 'NoneType' object has no attribute 'dim'

Plot the training loss (and validation loss/accuracy, if recorded).

## Testing your model
Using the previously created `DataLoader` for the test set, compute the percentage of correct predictions using the highest probability prediction. 

If your accuracy is over 70%, great work! 
This is a hard task to exceed 70% on.

If your accuracy is under 45%, you'll need to make improvements.
Go back and check your model architecture, loss function, and optimizer to make sure they're appropriate for an image classification task.

In [49]:
## YOUR CODE HERE ##

test_loss = 0
accuracy = 0
model.eval()
with torch.no_grad():
    
    for inputs, labels in test_dataloaders:
        inputs, labels = inputs.to(device), labels.to(device)
        logps = model.forward(inputs)
        batch_loss = criterion(logps, labels)
                    
        test_loss += batch_loss.item()
                    
         # Calculate accuracy
        ps = torch.exp(logps)
        top_p, top_class = ps.topk(1, dim=1)
        equals = top_class == labels.view(*top_class.shape)
        accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                    
    print(f"Test loss: {test_loss/len(test_dataloaders):.3f}.. "
              f"Test accuracy: {accuracy/len(test_dataloaders):.3f}")





AttributeError: 'NoneType' object has no attribute 'dim'

## Saving your model
Using `torch.save`, save your model for future loading.

In [51]:
## YOUR CODE HERE ##


checkpoint = {'input_size': 3072,
              'output_size': 64,
              'hidden_layers': [model.fc1],
              'state_dict': model.state_dict()}

#Using torch.save to save my model
torch.save(checkpoint, 'checkpointM.pth')
print("DONE")

DONE


## Make a Recommendation

Based on your evaluation, what is your recommendation on whether to build or buy? Explain your reasoning below.

Some things to consider as you formulate your recommendation:
* How does your model compare to Detectocorp's model?
* How does it compare to the far more advanced solutions in the literature? 
* What did you do to get the accuracy you achieved? 
* Is it necessary to improve this accuracy? If so, what sort of work would be involved in improving it?



**Double click this cell to modify it**



## Submit Your Project

When you are finished editing the notebook and are ready to turn it in, simply click the **SUBMIT PROJECT** button in the lower right.

Once you submit your project, we'll review your work and give you feedback if there's anything that you need to work on. If you'd like to see the exact points that your reviewer will check for when looking at your work, you can have a look over the project [rubric](https://review.udacity.com/#!/rubrics/3077/view).